# Project 1
## 1.
Objective: $Min \sum_{t=1}^{T} f_ty_t + \sum_{t=1}^{T} c_tx_t + \sum_{t=1}^{T} h_ts_t$

Subject to:
$d_t = x_t + s_{t-1} - s_t \qquad \forall t \in H$
$x_t \leq D_ty_t \qquad \forall t \in H$

$y_t \in [0,1]$
<br>
$x_t, s_t \geq 0$

Where:
$D_t = \sum_{i=t}^{T} d_i$
$s_0, s_T = 0$

In [81]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

# Import data and define constants

T = 12  # Num periods
rep = 2  # Tab number

tabs = {'6-1': '6-periods (1)',
    '6-2': '6-periods (2)',
    '12-1': '12-periods (1)',
    '12-2': '12-periods (2)',
    '24-1': '24-periods (1)',
    '24-2': '24-periods (2)',
    '52-1': '52-periods (1)',
    '52-2': '52-periods (2)',
    '104-1': '104-periods (1)',
    '104-2': '104-periods (2)'}

xls = pd.ExcelFile(r'../ULSP-instancesR.xlsx')

df = pd.read_excel(xls, sheet_name=tabs[f'{T}-{rep}'])

# Add zero row and adjust index to align with indicies prompt
df.loc[-1] = [0]*6
df.index = df.index + 1
df = df.sort_index()

# Constants
d = df['Demand Forecast']
f = df['Setup Cost']
c = df['Production cost']
h = df['Holding cost']
b = df['Backlogging cost']

D = [d[i:].sum() for i in range(len(d))]

# Set
H = range(1,T+1)
H_0 = range(T+1)

# Gurobi environment
ENV = gp.Env()
ENV.setParam('OutputFlag', 0)


Restricted license - for non-production use only - expires 2025-11-24


In [82]:
def extract_model_vars(x, y, s):
    # ---- Extract variable values ----
    x_values = np.zeros(len(H_0))
    y_values = np.zeros(len(H_0))
    s_values = np.zeros(len(H_0))

    for t in H:
        x_values[t] = x[t].X
        y_values[t] = y[t].X
        s_values[t] = s[t].X

    return x_values, y_values, s_values


def inequality_separation(x, y, s):
    S = {}
    v = {}
    Djl = {}
    for l in H:
        S[l] = set()
        for j in range(1,l+1):
            Djl[(j,l)] = d[j:l+1].sum()
            if x[j] - Djl[(j,l)]*y[j] > 0:
                S[l].add(j)
        v[l] = sum([x[j] - Djl[(j,l)]*y[j] if j in S[l] else 0]) - s[l] # TODO: confirm, error in problem 5, summation on 3rd line should be j \in S_l, not t \in S_l?
    return v, S, Djl


def run_model(relax=False, isp=False):
    start_time = time.time()

    # ---- Initiate model ----
    model = gp.Model(env=ENV)

    # ---- Decision variables ----

    # Units produced
    x = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='x')

    # Whether a lot is made
    if relax:
        y = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='y')
    else:
        y = model.addVars(T+1, vtype=GRB.BINARY, name='y')

    # Amount held in inventory at end of t
    s = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='s')


    # ---- Constraints ----

    # Initial values
    model.addConstr(s[0] == 0)
    model.addConstr(x[0] == 0)
    model.addConstr(y[0] == 0)

    # Material balance
    model.addConstrs(int(d[t]) == x[t] + s[t-1] - s[t] for t in H)

    # y definition
    model.addConstrs(x[t] <= D[t]*y[t] for t in H)


    # ---- Objective ----
    obj = gp.LinExpr()

    for t in H:
        obj.addTerms([f[t], c[t], h[t]], [y[t], x[t], s[t]])

    model.setObjective(obj, GRB.MINIMIZE)

    end_time = time.time()
    total_time = end_time - start_time

    # ---- Optimize ----
    model.optimize()

    # ---- Extract values ----
    x_values, y_values, s_values = extract_model_vars(x, y, s)

    if isp:
        if not relax:
            print("WARNING: isp is only designed to be used when relax=True")

        v, S, Djl = inequality_separation(x_values, y_values, s_values)

        count = 0
        while max(v.values())>0 and count <= 100:
            l_max = max(v.items(), key=lambda x: x[1])[0]
            # TODO: question, do I consider all S subsets from l = 0 to l, or just S[l]?
            model.addConstrs(gp.quicksum(x[t] for t in S[l]) <= gp.quicksum(Djl[(t,l)]*y[t] for t in S[l]) + s[l] for l in range(1,l_max+1))
            model.optimize()
            x_values, y_values, s_values = extract_model_vars(x, y, s)

            v, S, Djl = inequality_separation(x_values, y_values, s_values)
            # print("l_max: ", l_max)
            # print("v: ", v)
            # print("v_max: ", max(v.items(), key=lambda x: x[1]))
            if count == 99:
                print("WARNING: isp max iterations exceeded")
            count += 1

    return x_values, y_values, s_values, total_time

def print_model_results(relax, isp):
    x_values, y_values, s_values, total_time = run_model(relax=relax, isp=isp)
    print("x: ", x_values)
    print("y: ", y_values)
    print("s: ", s_values)
    print("total_time: ", total_time)

print("MILP model without relaxation:")
print_model_results(relax=False, isp=False)

print()
print("LP model with relaxation:")
print_model_results(relax=True, isp=False)

MILP model without relaxation:
x:  [  0. 623. 906.   0. 958. 319.   0. 689. 414. 372. 346.   0. 938.]
y:  [0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1.]
s:  [ 0.  0. 71.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
total_time:  0.0010554790496826172

LP model with relaxation:
x:  [  0. 623. 835.  71. 958. 319.   0. 689. 414. 372. 346.   0. 938.]
y:  [0.         0.11194969 0.16895994 0.01728756 0.23736373 0.10363873
 0.         0.24972816 0.2        0.22463768 0.2694704  0.
 1.        ]
s:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
total_time:  0.0011172294616699219


# 2.
Computation times are comparable at around 0.001s for the 6-element dataset and 0.006 for the 104-element dataset. Solutions for production quantities and holding are the same for the 6-element set, but not the 104-element set, although the y variable indicating whether a lot is produced is no longer strictly 0 or 1. If it is 0, no lot is produced, but when a lot is produced, y is a fractional value. This will not affect the production plan, but it will give an artificially lower objective.

## TODO: make code that automatically compares all variables for all datasets and gives a summary output?

# 3.
 Yes, we could use a heuristic here (i.e. dynamic programming) to get the same result as the MILP. In fact, this would be a good application for the Silver-Meal heuristic, which is similar to (Wagner-Whitin, 1958), except the former calculates cost per unit produced, and (Wagner-Whitin, 1958) looks at the absolute cost to fill the demand in a particular period.

## TODO: The question asks to identify a particular pattern?

In [83]:
# See https://www.youtube.com/watch?v=7vE-gm9qxpk

z = np.zeros(T+1)
x_ww = np.zeros(T+1)

for t in H:
    t_t = t
    z_t = np.zeros(t+1)
    while t_t > 0:

        z_t[t_t] = f[t_t]
        t_t -= 1

    z[t] = min(z_t)

# TODO: finish # 3

# 4.
This inequality states that the amount of units produced in any subset of periods must be less than or equal to the demand of those periods plus the ending inventory.
Indeed, the constraint for the definition of $y_t$ is:
$x_t \leq D_ty_t \qquad \forall t \in H$

If this is the case, then cumulative (i.e. summation of) $x_t$ must also be less than the cumulative $Dy_t$. $s_l$ is defined to be nonnegative, so adding this to the right side does not change the inequality.

# 5.

In [84]:
x, y, s, _ = run_model(relax=True)
v, S, Djl = inequality_separation(x, y, s)


print(v)
print(max(v.items(), key=lambda x: x[1]))


{1: 553.2553459119497, 2: 693.9184540671793, 3: 69.77258339420501, 4: 730.605550049554, 5: 285.93924626380766, 6: 0.0, 7: 516.9372961217832, 8: 331.2, 9: 288.4347826086956, 10: 252.76323987538942, 11: 0.0, 12: 0.0}
(4, 730.605550049554)


# 6.

In [85]:
print_model_results(relax=True, isp=True)

x:  [  0. 623. 906.   0. 958. 319.   0. 689. 414. 372. 346.   0. 938.]
y:  [0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1.]
s:  [ 0.  0. 71.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
total_time:  0.001100301742553711
